In [2]:
import sys

In [23]:
sys.path.append(r"C:\Users\Eddie\Documents\marl_sigctrl\framework")
sys.path.append(r"C:\Users\Eddie\Documents\marl_sigctrl\framework\signal_control\smartcross")
sys.path.append(r"C:\Users\Eddie\Documents\marl_sigctrl")

In [24]:
sys.path

['c:\\Users\\Eddie\\Documents\\marl_sigctrl\\signal_control\\smartcross\\envs',
 'D:\\sumo\\tools',
 'd:\\Anaconda\\python311.zip',
 'd:\\Anaconda\\DLLs',
 'd:\\Anaconda\\Lib',
 'd:\\Anaconda',
 '',
 'd:\\Anaconda\\Lib\\site-packages',
 'd:\\Anaconda\\Lib\\site-packages\\win32',
 'd:\\Anaconda\\Lib\\site-packages\\win32\\lib',
 'd:\\Anaconda\\Lib\\site-packages\\Pythonwin',
 'D:\\sumo\\tools',
 'C:\\Users\\Eddie\\Documents\\marl_sigctrl\\framework',
 'C:\\Users\\Eddie\\Documents\\marl_sigctrl\\framework',
 'C:\\Users\\Eddie\\Documents\\marl_sigctrl\\framework\\signal_control',
 'C:\\Users\\Eddie\\Documents\\marl_sigctrl\\framework',
 'C:\\Users\\Eddie\\Documents\\marl_sigctrl\\framework\\signal_control',
 'C:\\Users\\Eddie\\Documents\\marl_sigctrl\\framework',
 'C:\\Users\\Eddie\\Documents\\marl_sigctrl\\framework\\signal_control\\smartcross',
 'C:\\Users\\Eddie\\Documents\\marl_sigctrl\\framework',
 'C:\\Users\\Eddie\\Documents\\marl_sigctrl\\framework\\signal_control\\smartcross',
 '

In [110]:
import unittest
import numpy as np
from sklearn.cluster import KMeans
from easydict import EasyDict
from envs.sumo_env import SumoEnv  # 假设 SumoEnv 类在 sumo_env.py 文件中

class TestSumoEnv(unittest.TestCase):

    def setUp(self):
        # 创建一个 SumoEnv 实例，用于测试 _get_cls 方f
        cfg = {
            'sumocfg_path': 'sumo_3roads/rl_wj.sumocfg',
            "tls": ['ftddj_wjj', 'ftddj_frj', 'htddj_gsndj'],
            "max_episode_steps": 1500,
            "green_duration":10,
            "yellow_duration":3,
            "obs": {'obs_type': ['phase', 'lane_pos_vec'], 
                    'lane_grid_num': 10,
                    'traffic_volumn_ratio': 7.5,
                    'use_centralized_obs': False, 
                    'padding': True},
            "action": {'action_type': 'change', 'use_multi_discrete': True},
            "reward": {'use_centralized_reward': True, 'reward_type': {'queue_len': 1.0}},
            "collector_env_num": 1,
            "evaluator_env_num": 1,
            "agent_num": 3,
            "obs_dim": 42,
            "cls_num" :1,
        }
        
        cfg = EasyDict(cfg)
        self.env = SumoEnv(cfg)

    def test_get_cls(self):
        # 测试输入数据
        obs_n = np.random.rand(6, 42)  # 6 个智能体，每个智能体的观测维度为 42
        k = 2  # 聚类数量

        # 调用 _get_cls 方法
        neb = self.env._get_cls(obs_n, k)

        # 检查输出形状
        self.assertEqual(neb.shape, (k, self.env._agent_num, self.env._obs_dim))

        # 检查输出类型
        self.assertIsInstance(neb, np.ndarray)

        # 检查聚类标签是否正确
        coordinates = obs_n[:, 0:2]
        kmeans = KMeans(n_clusters=k, n_init=1).fit(coordinates)
        cluster_labels = kmeans.labels_

        for i, cluster in enumerate(cluster_labels):
            print(f"Agent {i} is in cluster {cluster}")
            self.assertTrue(np.array_equal(neb[cluster][i], obs_n[i]))

    def test_get_cls_with_zero_agents(self):
        # 测试输入数据
        obs_n = np.array([])  # 0 个智能体
        k = 2  # 聚类数量

        # 调用 _get_cls 方法
        neb = self.env._get_cls(obs_n, k)

        # 检查输出形状
        self.assertEqual(neb.shape, (k, 0, self.env._obs_dim))

        # 检查输出类型
        self.assertIsInstance(neb, np.ndarray)

    def test_get_cls_with_one_agent(self):
        # 测试输入数据
        obs_n = np.random.rand(1, 42)  # 1 个智能体，观测维度为 42
        k = 1  # 聚类数量

        # 调用 _get_cls 方法
        neb = self.env._get_cls(obs_n, k)

        # 检查输出形状
        self.assertEqual(neb.shape, (k, self.env._agent_num, self.env._obs_dim))

        # 检查输出类型
        self.assertIsInstance(neb, np.ndarray)

        # 检查聚类标签是否正确
        coordinates = obs_n[:, 0:2]
        kmeans = KMeans(n_clusters=k, n_init=1).fit(coordinates)
        cluster_labels = kmeans.labels_

        for i, cluster in enumerate(cluster_labels):
            self.assertTrue(np.array_equal(neb[cluster][i], obs_n[i]))

    def test_get_cls_with_multiple_clusters(self):
        # 测试输入数据
        obs_n = np.random.rand(6, 42)  # 6 个智能体，每个智能体的观测维度为 42
        k = 3  # 聚类数量

        # 调用 _get_cls 方法
        neb = self.env._get_cls(obs_n, k)

        # 检查输出形状
        self.assertEqual(neb.shape, (k, self.env._agent_num, self.env._obs_dim))

        # 检查输出类型
        self.assertIsInstance(neb, np.ndarray)

        # 检查聚类标签是否正确
        coordinates = obs_n[:, 0:2]
        kmeans = KMeans(n_clusters=k, n_init=1).fit(coordinates)
        cluster_labels = kmeans.labels_

        for i, cluster in enumerate(cluster_labels):
            self.assertTrue(np.array_equal(neb[cluster][i], obs_n[i]))

# if __name__ == '__main__':
#     unittest.main()

In [111]:
sumo_env_test = TestSumoEnv()
sumo_env_test.setUp()
sumo_env_test.test_get_cls()

d:\Anaconda\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


IndexError: index 3 is out of bounds for axis 0 with size 3

In [89]:
cfg["sumocfg_path"]

'sumo_3roads/rl_wj.sumocfg'